In [ ]:
!pip install hmmlearn -q

# About

This notebook implements the concept of calculating Remaining Useful Life (RUL) from the NASA bearing dataset. The paper references for implementing the concept are:
1. Tobon Mejia, Diego & Medjaher, Kamal & Zerhouni, Noureddine & Tripot, Gerard. (2010). [A Mixture of Gaussians Hidden Markov Model for failure diagnostic and prognostic](https://www.researchgate.net/publication/224177188_A_Mixture_of_Gaussians_Hidden_Markov_Model_for_failure_diagnostic_and_prognostic). 6th Annual IEEE Conference on Automation Science and Engineering, CASE'10.. 338 - 343. 10.1109/COASE.2010.5584759.
2. Medjaher, Kamal & Tobon Mejia, Diego & Zerhouni, Noureddine. (2012). [Remaining Useful Life Estimation of Critical Components With Application to Bearings](https://www.researchgate.net/publication/254059871_Remaining_Useful_Life_Estimation_of_Critical_Components_With_Application_to_Bearings). IEEE Transactions on Reliability - TR. 61. 292-302. 10.1109/TR.2012.2194175.
3. Tobon Mejia, Diego & Medjaher, Kamal & Zerhouni, Noureddine & Tripot, Gerard. (2012). [A Data-Driven Failure Prognostics Method Based on Mixture of Gaussians Hidden Markov Models](https://www.researchgate.net/publication/254059873_A_Data-Driven_Failure_Prognostics_Method_Based_on_Mixture_of_Gaussians_Hidden_Markov_Models). IEEE Transactions on Reliability - TR. 61. 491-503. 10.1109/TR.2012.2194177. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, random

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
sns.set_style('whitegrid')

from hmmlearn import hmm

# Description Dataset
The data was generated by the NSF I/UCR Center for Intelligent Maintenance Systems (IMS – www.imscenter.net) with support from Rexnord Corp. in Milwaukee, WI.

**Test Rig Setup**

Four bearings were installed on a shaft. The rotation speed was kept constant at 2000 RPM by an AC motor coupled to the shaft via rub belts. A radial load of 6000 lbs is applied onto the shaft and bearing by a spring mechanism. All bearings are force lubricated. Rexnord ZA-2115 double row bearings were installed on the shaft as shown in Figure 1. PCB 353B33 High Sensitivity Quartz ICP accelerometers were installed on the bearing housing (two accelerometers for each bearing [x- and y-axes] for data set 1, one accelerometer for each bearing for data sets 2 and 3). Sensor placement is also shown in Figure 1. All failures occurred after exceeding designed life time of the bearing which is more than 100 million revolutions.

![image.png](attachment:51495a34-c91d-4476-9aae-57525c5d357d.png)]

Figure 1 - Bearing Test Rig and sensor placement illustration (Qiu et al., 2006)

**Data Structure**

Three (3) data sets are included in the data packet (IMS-Rexnord Bearing Data.zip). Each data set describes a test-to-failure experiment. Each data set consists of individual files that are 1-second vibration signal snapshots recorded at specific intervals. Each file consists of 20,480 points with the sampling rate set at 20 kHz. The file name indicates when the data was collected. Each record (row) in the data file is a data point. Data collection was facilitated by NI DAQ Card 6062E. Larger intervals of time stamps (showed in file names) indicate resumption of the experiment in the next working day.

**Set No. 1**

| Index                    | Description                                                                                                                   |
|--------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| Recording Duration:      | October 22, 2003 12:06:24 to November 25, 2003 23:39:56                                                                       |
| No. of Files:            | 2,156                                                                                                                         |
| No. of Channels:         | 8                                                                                                                             |
| Channel Arrangement:     | Bearing 1 – Ch 1&2; Bearing 2 – Ch 3&4; Bearing 3 – Ch 5&6; Bearing 4 – Ch 7&8.                                               |
| File Recording Interval: | Every 10 minutes (except the first 43 files were taken every 5 minutes)                                                       |
| File Format:             | ASCII                                                                                                                         |
| Description:             | At the end of the test-to-failure experiment, inner race defect occurred in bearing 3 and roller element defect in bearing 4. |

**Set No. 2**

| Index                    | Description                                                                             |
|--------------------------|-----------------------------------------------------------------------------------------|
| Recording Duration:      | February 12, 2004 10:32:39 to February 19, 2004 06:22:39                                |
| No. of Files:            | 984                                                                                     |
| No. of Channels:         | 4                                                                                       |
| Channel Arrangement:     | Bearing 1 – Ch 1; Bearing2 – Ch 2; Bearing3 – Ch3; Bearing 4 – Ch 4.                    |
| File Recording Interval: | Every 10 minutes                                                                        |
| File Format:             | ASCII                                                                                   |
| Description:             | At the end of the test-to-failure experiment, outer race failure occurred in bearing 1. |

**Set No. 3**

| Index                    | Description                                                                             |
|--------------------------|-----------------------------------------------------------------------------------------|
| Recording Duration:      | March 4, 2004 09:27:46 to April 4, 2004 19:01:57                                        |
| No. of Files:            | 4,448                                                                                   |
| No. of Channels:         | 4                                                                                       |
| Channel Arrangement:     | Bearing 1 – Ch 1; Bearing2 – Ch 2; Bearing3 – Ch3; Bearing 4 – Ch 4.                    |
| File Recording Interval: | Every 10 minutes                                                                        |
| File Format:             | ASCII                                                                                   |
| Description:             | At the end of the test-to-failure experiment, outer race failure occurred in bearing 3. |

In [ ]:
SEED = 2021
np.random.seed(SEED)
random.seed(SEED)

For feature extraction, you can check this notebook https://www.kaggle.com/yasirabd/nasa-bearing-feature-extraction

In [ ]:
# load dataset
set1 = pd.read_csv('../input/nasa-bearing-time-features/set1_timefeatures.csv')
set1.columns = ['date'] + list(set1.columns[1:])
set1['date'] = pd.to_datetime(set1['date'])

set2 = pd.read_csv('../input/nasa-bearing-time-features/set2_timefeatures.csv')
set2.columns = ['date'] + list(set2.columns[1:])
set2['date'] = pd.to_datetime(set2['date'])

set3 = pd.read_csv('../input/nasa-bearing-time-features/set3_timefeatures.csv')
set3.columns = ['date'] + list(set3.columns[1:])
set3['date'] = pd.to_datetime(set3['date'])

# set date as index
set1 = set1.set_index('date')
set2 = set2.set_index('date')
set3 = set3.set_index('date')

set1.head()

In [ ]:
# merge a and b from bearing 1-4
set1['B1_mean'] = (set1['B1_a_mean'] + set1['B1_b_mean'])/2
set1['B1_std'] = (set1['B1_a_std'] + set1['B1_b_std'])/2
set1['B1_skew'] = (set1['B1_a_skew'] + set1['B1_b_skew'])/2
set1['B1_kurtosis'] = (set1['B1_a_kurtosis'] + set1['B1_b_kurtosis'])/2
set1['B1_entropy'] = (set1['B1_a_entropy'] + set1['B1_b_entropy'])/2
set1['B1_rms'] = (set1['B1_a_rms'] + set1['B1_b_rms'])/2
set1['B1_max'] = (set1['B1_a_max'] + set1['B1_b_max'])/2
set1['B1_p2p'] = (set1['B1_a_p2p'] + set1['B1_b_p2p'])/2

set1['B2_mean'] = (set1['B2_a_mean'] + set1['B2_b_mean'])/2
set1['B2_std'] = (set1['B2_a_std'] + set1['B2_b_std'])/2
set1['B2_skew'] = (set1['B2_a_skew'] + set1['B2_b_skew'])/2
set1['B2_kurtosis'] = (set1['B2_a_kurtosis'] + set1['B2_b_kurtosis'])/2
set1['B2_entropy'] = (set1['B2_a_entropy'] + set1['B2_b_entropy'])/2
set1['B2_rms'] = (set1['B2_a_rms'] + set1['B2_b_rms'])/2
set1['B2_max'] = (set1['B2_a_max'] + set1['B2_b_max'])/2
set1['B2_p2p'] = (set1['B2_a_p2p'] + set1['B2_b_p2p'])/2

set1['B3_mean'] = (set1['B3_a_mean'] + set1['B3_b_mean'])/2
set1['B3_std'] = (set1['B3_a_std'] + set1['B3_b_std'])/2
set1['B3_skew'] = (set1['B3_a_skew'] + set1['B3_b_skew'])/2
set1['B3_kurtosis'] = (set1['B3_a_kurtosis'] + set1['B3_b_kurtosis'])/2
set1['B3_entropy'] = (set1['B3_a_entropy'] + set1['B3_b_entropy'])/2
set1['B3_rms'] = (set1['B3_a_rms'] + set1['B3_b_rms'])/2
set1['B3_max'] = (set1['B3_a_max'] + set1['B3_b_max'])/2
set1['B3_p2p'] = (set1['B3_a_p2p'] + set1['B3_b_p2p'])/2

set1['B4_mean'] = (set1['B4_a_mean'] + set1['B4_b_mean'])/2
set1['B4_std'] = (set1['B4_a_std'] + set1['B4_b_std'])/2
set1['B4_skew'] = (set1['B4_a_skew'] + set1['B4_b_skew'])/2
set1['B4_kurtosis'] = (set1['B4_a_kurtosis'] + set1['B4_b_kurtosis'])/2
set1['B4_entropy'] = (set1['B4_a_entropy'] + set1['B4_b_entropy'])/2
set1['B4_rms'] = (set1['B4_a_rms'] + set1['B4_b_rms'])/2
set1['B4_max'] = (set1['B4_a_max'] + set1['B4_b_max'])/2
set1['B4_p2p'] = (set1['B4_a_p2p'] + set1['B4_b_p2p'])/2

set1 = set1[['B1_mean','B1_std','B1_skew','B1_kurtosis','B1_entropy','B1_rms','B1_max','B1_p2p',
             'B2_mean','B2_std','B2_skew','B2_kurtosis','B2_entropy','B2_rms','B2_max','B2_p2p',
             'B3_mean','B3_std','B3_skew','B3_kurtosis','B3_entropy','B3_rms','B3_max','B3_p2p',
             'B4_mean','B4_std','B4_skew','B4_kurtosis','B4_entropy','B4_rms','B4_max','B4_p2p']]
set1.head()

# Slice Features (Mean, RMS, Skewness, Kurtosis)

We only used 4 time features: mean, RMS, skewness, and kurtosis.

References: (Tobon, 2010) [A mixture of gaussians hidden markov model for failure diagnostic and prognostic.](https://hal.archives-ouvertes.fr/hal-00525073/documenthttps://hal.archives-ouvertes.fr/hal-00525073/document)

In [ ]:
cols = ['B1_mean','B1_rms','B1_skew','B1_kurtosis', 
        'B2_mean','B2_rms','B2_skew','B2_kurtosis',
        'B3_mean','B3_rms','B3_skew','B3_kurtosis',
        'B4_mean','B4_rms','B4_skew','B4_kurtosis',]
set1 = set1[cols]
set2 = set2[cols]
set3 = set3[cols]

In [ ]:
# statistics
set1.describe().T

In [ ]:
# statistics
set2.describe().T

In [ ]:
# statistics
set3.describe().T

In [ ]:
def plot_features(df):
    fig, axes = plt.subplots(4, 1, figsize=(15, 5*4))
    
    axes[0].plot(df['B1_mean'])
    axes[0].plot(df['B2_mean'])   
    axes[0].plot(df['B3_mean'])
    axes[0].plot(df['B4_mean'])
    axes[0].legend(['B1','B2','B3','B4'])
    axes[0].set_title('Mean')
    
    axes[1].plot(df['B1_rms'])
    axes[1].plot(df['B2_rms'])   
    axes[1].plot(df['B3_rms'])
    axes[1].plot(df['B4_rms'])
    axes[1].legend(['B1','B2','B3','B4'])
    axes[1].set_title('RMS')
    
    axes[2].plot(df['B1_skew'])
    axes[2].plot(df['B2_skew'])   
    axes[2].plot(df['B3_skew'])
    axes[2].plot(df['B4_skew'])
    axes[2].legend(['B1','B2','B3','B4'])
    axes[2].set_title('Skewness')
    
    axes[3].plot(df['B1_kurtosis'])
    axes[3].plot(df['B2_kurtosis'])   
    axes[3].plot(df['B3_kurtosis'])
    axes[3].plot(df['B4_kurtosis'])
    axes[3].legend(['B1','B2','B3','B4'])
    axes[3].set_title('Kurtosis')

In [ ]:
# plot set 1
plot_features(set1)

In [ ]:
# plot set 2
plot_features(set2)

In [ ]:
# plot set 3
plot_features(set3)

# Scaling

As mentioned (1), we don't need to perform feature scaling.

But, after I did some experiments the result is better with feature scaling.

And in hmmlearn library (2), they use KMeans to cluster the number of components in data. Also, KMeans is sensitive with the variance of data, so it is better to perform feature scaling.

- (1): https://stats.stackexchange.com/questions/371333/is-it-important-to-make-a-feature-scaling-before-using-gaussian-mixture-model
- (2): https://github.com/hmmlearn/hmmlearn/blob/master/lib/hmmlearn/hmm.py

In [ ]:
def slice_columns(columns, target='B1'):
    if target == 'B1':
        return columns[0:4]
    elif target == 'B2':
        return columns[4:8]
    elif target == 'B3':
        return columns[8:12]
    elif target == 'B4':
        return columns[12:]

In [ ]:
# with scikit-learn
from sklearn.preprocessing import MinMaxScaler, StandardScaler

set1_scaled = set1.copy()
set2_scaled = set2.copy()
set3_scaled = set3.copy()

ss_l = []
minmax_l = []

# scaling set test 1
for bear in ['B1','B2','B3','B4']:
    col_features = slice_columns(set1.columns, target=bear)
    scaler = StandardScaler()
#     scaler = MinMaxScaler(feature_range=(-2,2))
    set1_scaled[col_features] = scaler.fit_transform(set1[col_features])
    ss_l.append(scaler)
#     minmax_l.append(scaler)
    
# scaling set test 2
for bear in ['B1','B2','B3','B4']:
    col_features = slice_columns(set2.columns, target=bear)
    scaler = StandardScaler()
#     scaler = MinMaxScaler(feature_range=(-2,2))
    set2_scaled[col_features] = scaler.fit_transform(set2[col_features])
    ss_l.append(scaler)
#     minmax_l.append(scaler)

# scaling set test 3, except bearing 3
for bear in ['B1','B2','B4']:
    col_features = slice_columns(set3.columns, target=bear)
    scaler = StandardScaler()
#     scaler = MinMaxScaler(feature_range=(-2,2))
    set3_scaled[col_features] = scaler.fit_transform(set3[col_features])#.round(4)
    ss_l.append(scaler)
#     minmax_l.append(scaler)

In [ ]:
# before and after scaling
fig, axes = plt.subplots(1, 2, figsize=(14,5), dpi=80)
axes[0].plot(set1['B1_mean'])
axes[1].plot(set1_scaled['B1_mean'])

axes[0].set_title('Before scaling (Mean)')
axes[1].set_title('After scaling (Mean)');

# Prepare data

- Testing: Set test 3 bearing 3
    - 'S3_B3'
- Learning: 11 sensor data from each set test
    - 'S1_B1','S1_B2','S1_B3','S1_B4',
    - 'S2_B1','S2_B2','S2_B3','S2_B4',
    - 'S3_B1','S3_B2','S3_B4'

# GMMHMM

In [ ]:
def flip_transmat(tm, ix_sort):
    tm_ = tm.copy()
    for i,ix in enumerate(ix_sort):
        tm_[i, :] = tm[ix[0], :]
    tm__ = tm_.copy()
    for i,ix in enumerate(ix_sort):
        tm__[:, i] = tm_[:,ix[0]]
    return tm__

**How to decide the covariance type?**

Covariance matrices for Gaussian Mixture Model has several types such as Full, Tied, Diagonal, and Spherical. You can read more details on this article (1). 

If we look up into the means of the data features (after scaling) for each state in GMM model, they have close value between each other. So, it is decided that covariance type `tied` has potentital to differentiate between each state. (Still need to understand the concept of this theory)

![image.png](attachment:37015e9d-5b95-4acb-9b44-b22fbefbef39.png)


- (1) https://stats.stackexchange.com/questions/326671/different-covariance-types-for-gaussian-mixture-models/326678#326678

**How to make a left-right GMMHMM model?**

![image.png](attachment:d6863120-2ff8-4c07-a139-be1f0c95cb11.png)

Based on references from Sequentia library (1)(2)(3) and hmmlearn library (4), we need to:

1. First initiate `start probability` randomly and `transition matrix` with upper right values.
2. In GMMHMM model, 
    - define `init_params` equal `cmw`, means only initialize covariance, means, and weights. Since, we will initiate start probability (`s`) and transition matrix (`t`) manually.
    - define `params` equal `stcmw`, means it will update those parameters when training.
3. Define `model.startprob_` and `model.transmat_` with its value which has been initialized before.


- (1) https://sequentia.readthedocs.io/en/latest/sections/classifiers/gmmhmm.html
- (2) https://github.com/eonu/sequentia/blob/master/lib/sequentia/classifiers/hmm/gmmhmm.py
- (3) https://github.com/eonu/sequentia/blob/master/lib/sequentia/classifiers/hmm/topologies/left_right.py
- (4) https://hmmlearn.readthedocs.io/en/latest/tutorial.html#building-hmm-and-generating-samples

Notes:
- Still need to modify the model into the left-to-right model. But, somehow when I defined the model as left-to-right, some of the training processes were not successful.

In [ ]:
#https://untitledtblog.tistory.com/133 : gaussian mixture model
#https://untitledtblog.tistory.com/97 : hidden markov model
def random_transitions(n_states) -> np.ndarray:
    """Sets the transition matrix as random (random probability of transitioning
    to all other possible states from each state) by sampling probabilities
    from a Dirichlet distribution, according to the topology.
    Returns
    -------
    transitions: :class:`numpy:numpy.ndarray` (float)
        The random transition matrix of shape `(n_states, n_states)`.
    """
    transitions = np.zeros((n_states, n_states))
    for i, row in enumerate(transitions):
        row[i:] = np.random.dirichlet(np.ones(n_states - i))
    return transitions

def create_gmmhmm():
    startprob = np.array([1., 0., 0.], dtype=np.float64)
    transmat = np.array([[0.9995, 0.0005,  0.],
                         [0.,     0.9998,  0.0002],
                         [0.,     0.,      1.0]], dtype=np.float64)
#     transmat = np.array([[0.38903512, 0.28715641, 0.32380848],
#                          [0.        , 0.56796488, 0.43203512],
#                          [0.        , 0.        , 1.        ]], dtype=np.float64)
#     transmat = random_transitions(n_states=3)
    
    model = hmm.GMMHMM(n_components=3, 
                       n_mix=2, 
                       covariance_type="tied", 
                       n_iter=1000, 
                       tol=1e-6,
#                        startprob_prior=startprob,
#                        transmat_prior=transmat,
#                        init_params='cmw',
#                        params='stcmw',
                       random_state=SEED,
                       verbose=False)
    model.n_features = 4
    model.startprob_ = startprob
    model.transmat_ = transmat
    
    return model

def fit_gmmhmm(model, data):
    
    # train model
    model.fit(data)

    # clasify each observation as state (0, 1, 2)
    hidden_states = model.predict(data)
    
    # get parameters of GMMHMM
    startprob_ = model.startprob_
    means_ = model.means_
    transmat_ = model.transmat_
    covars_ = model.covars_
    weights_ = model.weights_
    
    # reorganize by mean, so the the order of the states from lower to higher
    ix_sort = np.argsort(np.array([[np.mean(m)] for m in means_]), axis=0)
    
    hidden_states = np.array([ix_sort[st][0] for st in hidden_states])
    startprob = np.array([startprob_[ix][0] for ix in ix_sort])
    means = np.array([means_[ix][0] for ix in ix_sort])
    transmat = flip_transmat(transmat_, ix_sort)
    covars = np.array([covars_[ix][0] for ix in ix_sort])
    weights = np.array([weights_[ix][0] for ix in ix_sort])
    
    model.startprob_ = startprob
    model.means_ = means
    model.transmat_ = transmat
    model.covars_ = covars
    model.weights_ = weights
    
    # logprob
    logprob = model.score(data)
    
    return ix_sort, logprob, model, hidden_states

In [ ]:
def plot_rms_and_state(rms, state):
    fig, ax1 = plt.subplots(figsize=(12,5))
    ax2 = ax1.twinx()
    ax1.plot(rms)
    ax1.set_ylabel('RMS')
    ax1.set_xlabel('time (minutes)')
    ax2.plot(state, color='red')

    ax2.set_yticks(range(0,3))
    ax2.set_ylabel('state', rotation=270, labelpad=20)
    plt.show();

In [ ]:
# example on bearing 1 in the test 1

# initialize gmmhmm
gmmhmm = create_gmmhmm()

# setup data
col_features = slice_columns(set1_scaled.columns, target='B1')
data = set1_scaled[col_features]

# train gmmhmm
ix_sort, logprob, model, hidden_states = fit_gmmhmm(gmmhmm, data)

print(f'Log probability: {np.around(logprob, decimals=4)}')
print(f'Start probability: {np.around(model.startprob_, decimals=4)}')
print(f'Means:\n{np.around(model.means_, decimals=4)}')
print(f'Transition Matrix:\n{np.around(model.transmat_, decimals=3)}')
print(f'Covariance matrix:\n{np.around(model.covars_, decimals=4)}')
print(f'Weights:\n{np.around(model.weights_, decimals=4)}')
print(ix_sort)

# plot RMS and state
plot_rms_and_state(data[data.columns[1]].values, hidden_states)

## Training

In [ ]:
%%time
# model GMMHMM
model_gmmhmm = []

# train set test 1
for bear in ['B1','B2','B3','B4']:
    gmmhmm = create_gmmhmm()  # create model
    col_features = slice_columns(set1_scaled.columns, target=bear)
    ix_sort, logprob, model, hidden_states = fit_gmmhmm(gmmhmm, set1_scaled[col_features])  # train
    
    # append model into list
    model_gmmhmm.append((ix_sort, logprob, model, hidden_states))
    
# train set test 2
for bear in ['B1','B2','B3','B4']:
    gmmhmm = create_gmmhmm()  # create model
    col_features = slice_columns(set2_scaled.columns, target=bear)
    ix_sort, logprob, model, hidden_states = fit_gmmhmm(gmmhmm, set2_scaled[col_features])  # train
    
    # append model into list
    model_gmmhmm.append((ix_sort, logprob, model, hidden_states))
    
# train set test 3, except bearing 3
for bear in ['B1','B2','B4']:
    gmmhmm = create_gmmhmm()  # create model
    col_features = slice_columns(set3_scaled.columns, target=bear)
    ix_sort, logprob, model, hidden_states = fit_gmmhmm(gmmhmm, set3_scaled[col_features])  # train
    
    # append model into list
    model_gmmhmm.append((ix_sort, logprob, model, hidden_states))

In [ ]:
# get rms data for each bearing test, except set test 3 bearing 3 (S3_B3) because test set
rms_data = []
for bear in ['B1','B2','B3','B4']:
    col_features = slice_columns(set1_scaled.columns, target=bear)
    data = set1[col_features]
    rms = data[data.columns[1]]
    rms_data.append(rms)
for bear in ['B1','B2','B3','B4']:
    col_features = slice_columns(set2_scaled.columns, target=bear)
    data = set2[col_features]
    rms = data[data.columns[1]]
    rms_data.append(rms)
for bear in ['B1','B2','B4']:
    col_features = slice_columns(set3_scaled.columns, target=bear)
    data = set3[col_features]
    rms = data[data.columns[1]]
    rms_data.append(rms)

# sequence data for training
model_data = ['S1_B1','S1_B2','S1_B3','S1_B4',
              'S2_B1','S2_B2','S2_B3','S2_B4',
              'S3_B1','S3_B2','S3_B4']

for (ix_sort,logprob,model,hidden_states),data,rms in zip(model_gmmhmm, model_data, rms_data):
    print(f'Sequence data: {data}')
    print(f'Logprob\n {np.around(logprob, decimals=4)}')
    print(f'Start Probability {np.around(model.startprob_, decimals=4)}')
    print(f'Means\n {np.around(model.means_, decimals=4)}')
    print(f'Transition Matrix\n {np.around(model.transmat_, decimals=3)}')
    print(f'Mixture Weights\n {np.around(model.weights_, decimals=4)}')
    print(f'Covariance matrix:\n{np.around(model.covars_, decimals=4)}')

    # plot RMS and state
    plot_rms_and_state(rms.values, hidden_states)
    print()

## Testing (S3_B3)

In [ ]:
# logprob and plot decode state on S3_B3
logprob_l = []
i = 0
for (ix,logprob,model,hidden_states),data in zip(model_gmmhmm, model_data):
    # select features
    col_features = slice_columns(set3.columns, target='B3')
    
    # scaling
    S3_B3 = ss_l[i].transform(set3[col_features])
    i += 1
    
    # calculate logprob on model sample
    logprob = model.score(S3_B3)
    pred = model.predict(S3_B3)
    
    logprob_l.append(logprob)

    print(f'GMMHMM model from {data} got log probability on S3_B3: {logprob}')
    rms = set3[col_features]['B3_rms']
    
    # plot RMS and decode state
    plot_rms_and_state(rms.values, pred)
    print()

**How to interpret log probability?**

From reference (1), the more negative value of log probability, the better performance of the model. It happens because if we do exponentials of the log probability on the most negative log prob, the result will more close to zero.

But, if we see the decoding state of all the models, the result is not as expected from the theory. Theoretically, the GMMHMM model from `S1_B1` with log prob -21625910.26 is the best among others. But, the models from `S1_B4`, `S2_B1`, and `S3_B1` have better results on decoding states despite their log prob respectively got -428826.79, -44063.25, -106197.6555733793.

- (1) https://www.mathworks.com/matlabcentral/answers/351766-how-to-interpret-log-probability-from-hmmdecode

In [ ]:
# example exponential on negative log probability
np.exp(-100), np.exp(-150)

**What is the best model chosen?**

Comparing the result above, model from `S2_B1` is the best on predicting/decoding state of test data `S3_B3`.

In [ ]:
# index best model
i = 4

# select best model
ix, logprob, model, hidden_states = model_gmmhmm[i]

# transform data
S3_B3 = ss_l[i].transform(set3[col_features])
model.score(S3_B3)

# Remaining Useful Life (RUL)

Based on reference (1), the formula to calculate mean and standard deviation are:

$$\mu(D(S_i)) = \frac{\sum_{w = 1}^{\Omega} D(S_{iw})}{\Omega}$$

$$\sigma(D(S_i)) = \frac{\sum_{w = 1}^{\Omega} [D(S_{iw})- \mu(D(S_i))]^2}{\Omega}$$

where,
- $D(.)$ is visit duration
- $i$ is the state index
- $w$ is the visit index
- $\Omega$ is the total of visits

The formula to calculate RUL with $\eta$ is confidence value:

$$RUL_{upper} = \sum_{i = current state}^{N} [\mu(D(S_{i})) + \eta.\sigma(D(S_i))]$$

$$RUL_{mean} = \sum_{i = current state}^{N} \mu(D(S_{i}))$$

$$RUL_{lower} = \sum_{i = current state}^{N} [\mu(D(S_{i})) - \eta.\sigma(D(S_i))]$$

- (1) Tobon Mejia, Diego & Medjaher, Kamal & Zerhouni, Noureddine & Tripot, Gerard. (2010). [A Mixture of Gaussians Hidden Markov Model for failure diagnostic and prognostic](https://www.researchgate.net/publication/224177188_A_Mixture_of_Gaussians_Hidden_Markov_Model_for_failure_diagnostic_and_prognostic). 6th Annual IEEE Conference on Automation Science and Engineering, CASE'10.. 338 - 343. 10.1109/COASE.2010.5584759.

To align the formula and the code, I will try to implement figure of decoding state from set test 1 bearing from reference (1) above.
![image.png](attachment:beb76006-5643-49e1-bd94-16495e369588.png)

After did subtitution, we got duration for each state index.
![e952fec8-bf58-4cbb-b50e-488bd79e9b07.jfif](attachment:faabc8f9-effc-4f65-9e7f-71792a877194.jfif)

In [ ]:
def calculate_mean(durations):
    return np.mean(durations)

def calculate_std(durations):
    return np.std(durations)

In [ ]:
import math

D_S1 = [0.458, 0.286]
D_S2 = [0.393, 0.334]
D_S3 = [0.357, 0.328]

print(f"S1 = [mean(S1), std(S1)] = [{math.floor(calculate_mean(D_S1)*10000)}, {math.floor(calculate_std(D_S1)*10000)}]")
print(f"S2 = [mean(S2), std(S2)] = [{math.floor(calculate_mean(D_S2)*10000)}, {math.floor(calculate_std(D_S2)*10000)}]")
print(f"S3 = [mean(S3), std(S3)] = [{math.floor(calculate_mean(D_S3)*10000)}, {math.ceil(calculate_std(D_S3)*10000)}]")

The result is the same with the paper, so the code implemented is proven.

![image.png](attachment:a9af1e20-1591-4622-9d72-b66cd1721cd1.png)

After we've found mean and standard deviation for each state, we can calculate RUL from the formula above. 

The path estimation of the example on set test 1 bearing 1 above like this

```
S2 -> S1 -> S2 -> S3 -> S1 -> S3
```

In [ ]:
result = {'S1': {'mean': int(calculate_mean(D_S1)*10000),
                 'std': int(calculate_std(D_S1)*10000)},
          'S2': {'mean': int(calculate_mean(D_S2)*10000),
                 'std': int(calculate_std(D_S2)*10000)},
          'S3': {'mean': int(calculate_mean(D_S3)*10000),
                 'std': math.ceil(calculate_std(D_S3)*10000)}
         }
path = ['S2','S1','S2','S3','S1','S3']

result, path

In [ ]:
def calculate_rul(path, result, conf=0.95):
    rul = []
    
    cum_rul = 0
    for p in path:
        if p == 'S1':
            rul_upper = result.get('S1').get('mean') + conf * result.get('S1').get('std') + cum_rul
            rul_mean = result.get('S1').get('mean') + cum_rul
            rul_lower = result.get('S1').get('mean') - conf * result.get('S1').get('std') + cum_rul
            
            cum_rul += result.get('S1').get('mean')
        elif p == 'S2':
            rul_upper = result.get('S2').get('mean') + conf * result.get('S2').get('std') + cum_rul
            rul_mean = result.get('S2').get('mean') + cum_rul
            rul_lower = result.get('S2').get('mean') - conf * result.get('S2').get('std') + cum_rul
            
            cum_rul += result.get('S2').get('mean')
        elif p == 'S3':
            rul_upper = result.get('S3').get('mean') + conf * result.get('S3').get('std') + cum_rul
            rul_mean = result.get('S3').get('mean') + cum_rul
            rul_lower = result.get('S3').get('mean') - conf * result.get('S3').get('std') + cum_rul
            
            cum_rul += result.get('S3').get('mean')
        
        rul.append((rul_upper, rul_mean, rul_lower))
    return rul

In [ ]:
calculate_rul(path, result)

In [ ]:
range_index = [(0, 3930), (3930, 8510), (8510, 11850), (11850, 15420), (15420, 18280), (18280, 21560)]

In [ ]:
df_visual = pd.DataFrame()
df_visual['index'] = [i for i in range(0,21560,10)]
df_visual['state'] = [1]*393 + [0]*458 + [1]*334 + [2]*357 + [0]*286 + [2]*328
df_visual['rul_upper'] = [3915.25]*393 + [8172.0]*458 + [11270.25]*334 + [14552.75]*357 + [18952.0]*286 + [21697.75]*328
df_visual['rul_mean'] = [3635]*393 + [7355]*458 + [10990]*334 + [14415]*357 + [18135]*286 + [21560]*328
df_visual['rul_lower'] = [3354.75]*393 + [6538]*458 + [10709.75]*334 + [14277.25]*357 + [17318]*286 + [21422.25]*328
df_visual['rul_error_upper'] = (21560 - df_visual['rul_upper'])/21560 * 100
df_visual['rul_error_mean'] = (21560 - df_visual['rul_mean'])/21560 * 100
df_visual['rul_error_lower'] = (21560 - df_visual['rul_lower'])/21560 * 100

df_visual

In [ ]:
# hidden state
plt.figure(figsize=(12,5), dpi=80)
plt.plot(df_visual['index'], df_visual['state'])
plt.yticks(range(0,3));

In [ ]:
# RUL estimation
plt.figure(figsize=(12,8), dpi=80)
plt.plot(df_visual['index'], [21560]*2156, color='r', linestyle='--')
plt.plot(df_visual['index'], df_visual['rul_upper'], linestyle='--')
plt.plot(df_visual['index'], df_visual['rul_mean'])
plt.plot(df_visual['index'], df_visual['rul_lower'], linestyle='--')

plt.legend(['Real','RUL Upper', 'RUL Mean', 'RUL Lower'])
plt.ylabel('Failure time (min)')
plt.xlabel('Current time (min)');

In [ ]:
# RUL error associated
plt.figure(figsize=(12,8), dpi=80)
plt.plot(df_visual['index'], df_visual['rul_error_upper'], linestyle='--')
plt.plot(df_visual['index'], df_visual['rul_error_mean'])
plt.plot(df_visual['index'], df_visual['rul_error_lower'], linestyle='--')

plt.legend(['RUL Upper', 'RUL Mean', 'RUL Lower'])
plt.ylabel('Error (%)')
plt.xlabel('Current time (min)');

In [ ]:
# def rul2(time_state, conf_score):
#     mean = {0: [], 1: [], 2: []}
#     std = {0: [], 1: [], 2: []}
    
#     for data in time_state.items():
#         print(data)

In [ ]:
# # function calculate RUL
# def rul(time_state, conf):
#     # calculate mean and standard deviation
#     mean_std = {0: [], 1: [], 2: []}
#     for data in time_state.items():
#         state, time = data[0], data[1]
        
#         decrease_time = []
#         if time:
#             for t in time:
#                 decrease_time.append(t[1]-t[0])
            
#         mean_state = np.mean(decrease_time)
#         std_state = np.std(decrease_time)
        
#         mean_std[state].append((mean_state, std_state))
    
    
    
#     # convert nan to zero
#     mean_std[0][0], mean_std[1][0], mean_std[2][0] = np.nan_to_num(mean_std[0][0]), np.nan_to_num(mean_std[1][0]), np.nan_to_num(mean_std[2][0])

#     # rul upper
#     rul_upper = (mean_std[0][0][0] + conf * mean_std[0][0][1]) + \
#                 (mean_std[1][0][0] + conf * mean_std[1][0][1]) + \
#                 (mean_std[2][0][0] + conf * mean_std[2][0][1])

#     # RUL Mean
#     rul_mean = (mean_std[0][0][0]) + \
#                (mean_std[1][0][0]) + \
#                (mean_std[2][0][0])

#     # RUL lower
#     rul_lower = (mean_std[0][0][0] - conf * mean_std[0][0][1]) + \
#                 (mean_std[1][0][0] - conf * mean_std[1][0][1]) + \
#                 (mean_std[2][0][0] - conf * mean_std[2][0][1])
#     # print(mean_std)
#     return rul_upper, rul_mean, rul_lower

In [ ]:
# # model terbaik dari S3_B1
# col_features = slice_columns(set3.columns, target='B3')
# S3_B3 = minmax_l[0].transform(set3[col_features])

# # ambil model S3_B1
# gmmhmm = model_gmmhmm[0][2]

# x = range(len(S3_B3))
# y = gmmhmm.predict(S3_B3)

In [ ]:
# gmmhmm.transmat_

In [ ]:
# # slicing only for experiment
# x_ = range(len(S3_B3))[:150]
# y_ = gmmhmm.predict(S3_B3)[:150]
# y_

In [ ]:
# # calculate RUL
# time_state = {0: [], 1: [], 2: []}
# state_rul = []
# time_rul = []

# previous = None
# start_time = 0

# i = 0
# for time_x, state_y in zip(x, y):
#     # initiate start time and state
#     if previous is None:
#         previous = state_y
#         start_time = time_x
    
#     # if previous state different with next state, calculate time
#     if previous != state_y:       
#         time_state[previous].append((start_time, time_x))
#         start_time = time_x
#         rul_upper, rul_mean, rul_lower = rul(time_state, 0.95)
        
# #         print(time_state)
# #         print(f'RUL UPPER: {rul_upper}; RUL MEAN: {rul_mean}; RUL LOWER: {rul_lower}')
# #         print()
        
#         state_rul.append((rul_upper, rul_mean, rul_lower))
#         time_rul.append([time_x, rul_upper, rul_mean, rul_lower])
    
#     previous = state_y
    
#     # if time_x equals last time in index 
#     if time_x == x[-1]:
#         time_state[previous].append((start_time, time_x))  
#         rul_upper, rul_mean, rul_lower = rul(time_state, 0.95)
#         state_rul.append((rul_upper, rul_mean, rul_lower))
#         time_rul.append([time_x, rul_upper, rul_mean, rul_lower])
# # time_rul

In [ ]:
# df_rul = pd.DataFrame(data={'RUL UPPER': [], 'RUL MEAN': [], 'RUL LOWER': []})
# for data in state_rul:
#     df_rul = df_rul.append({'RUL UPPER': data[0], 'RUL MEAN': data[1], 'RUL LOWER': data[2]}, ignore_index=True)
# df_rul

In [ ]:
# df_time_rul = pd.DataFrame(time_rul, columns=['time', 'rul_upper', 'rul_mean', 'rul_lower'])
# df_time_rul

In [ ]:
# df_visualize_rul = pd.DataFrame({"time": [], "rul_upper": [], "rul_mean": [], "rul_lower": []})

# for index, data in df_time_rul.iterrows():
#     # if last index
#     if index == df_time_rul.shape[0]-1:
#         df_visualize_rul = df_visualize_rul.append({"time": data["time"], "rul_upper": data["rul_upper"], "rul_mean": data["rul_mean"], "rul_lower": data["rul_lower"]}, ignore_index=True)
#     else:
#         # check time for next index, if difference not 10, do something
#         current_time = data["time"]
#         next_time = df_time_rul.iloc[index+1]["time"]
#         if next_time - current_time > 10:
#             while next_time > current_time:
#                 df_visualize_rul = df_visualize_rul.append({"time": int(current_time), "rul_upper": data["rul_upper"], "rul_mean": data["rul_mean"], "rul_lower": data["rul_lower"]}, ignore_index=True)
#                 current_time += 10
#         else:
#             df_visualize_rul = df_visualize_rul.append({"time": int(current_time), "rul_upper": data["rul_upper"], "rul_mean": data["rul_mean"], "rul_lower": data["rul_lower"]}, ignore_index=True)

In [ ]:
# df_visualize_rul

In [ ]:
# plt.figure(figsize=(12,5), dpi=80)
# plt.plot(df_visualize_rul["time"], df_visualize_rul["rul_upper"])
# plt.plot(df_visualize_rul["time"], df_visualize_rul["rul_mean"])
# plt.plot(df_visualize_rul["time"], df_visualize_rul["rul_lower"])

# plt.title("Grafik Remaining Useful Life (RUL)", fontsize=20)
# plt.ylabel("Remaining Useful Life (RUL)", fontsize=16)
# plt.xlabel("Waktu (Menit)", fontsize=18)
# plt.legend(['RUL UPPER', 'RUL MEAN', 'RUL LOWER'], loc='upper left')
# plt.show()